In [17]:
!pip install squarify

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [18]:
!pip install underthesea

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [19]:
!pip install gensim

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import pandas as pd
import numpy as np
import re
from underthesea import word_tokenize
from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *

In [5]:
# Calculate average values for each GMM_segment, and return a size of each segment 
def create_df_agg(df, groupby_col):
    df.R = df.R.astype(int)
    df.F = df.F.astype(int)
    df.M = df.M.astype(int)
    
    df_agg = df.groupby(groupby_col).agg({
      'Recency': 'mean',
      'Frequency': 'mean',
      'Monetary': 'mean',
      'R': 'mean',
      'F': 'mean',
      'M': ['mean', 'count']}).round(0)

    df_agg.columns = df_agg.columns.droplevel()
    df_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'RMean','FMean','MMean','Count']
    df_agg['Percent'] = round((df_agg['Count']/df_agg.Count.sum())*100, 2)

    # Reset the index
    df_agg = df_agg.reset_index()
    return df_agg

In [6]:
# Visualization - Treemap
def treemap_customer_segmentation(df_agg):
    #Create our plot and resize it.
    fig = plt.gcf()
    ax = fig.add_subplot()
    fig.set_size_inches(22, 9)

    # create a color palette, mapped to these values
    cmap = matplotlib.cm.plasma
    norm = matplotlib.colors.Normalize(vmin=min(df_agg['Count']), vmax=max(df_agg['Count']))
    colors = [cmap(norm(value)) for value in df_agg['Count']]

    squarify.plot(sizes=df_agg['Count'],
                text_kwargs={'fontsize':8,'weight':'bold', 'fontname':"sans serif"},
                color=colors,
                label=['{} \n{:.0f} days \n{:.0f} orders \n{:.0f} $ \n{:.0f} customers ({}%)'.format(*df_agg.iloc[i])
                        for i in range(0, len(df_agg))], alpha=0.5 )


    plt.title("Customers Segments",fontsize=26,fontweight="bold")
    plt.axis('off')

    # plt.savefig('RFM Segments.png')
    plt.show()

In [7]:
def univariate_cont_analysis(vars,df):
    results = []
    for var in vars:
        mean = df[var].mean()
        median = df[var].median() 
        mode = df[var].mode()
        max_val = df[var].max()
        min_val = df[var].min()
        range_val = np.ptp(df[var])
        variance = df[var].var()
        skewness = df[var].skew()
        kurtosis = df[var].kurtosis()
        result = [var, mean,median,mode,max_val,min_val,range_val,variance,skewness,kurtosis]
        results.append(result)
        df_result = pd.DataFrame(results, columns=['var_name','mean','median','mode','max_val','min_val',
                                                  'range_val','variance','skewness','kurtosis'],
                                ).set_index('var_name')
    return df_result

In [8]:
#Visualization - distplot
def visualization_distplot(vars,df):
    plt.figure(figsize=(15,4))
    for i in range(1,len(vars)+1):
        plt.subplot(1,3,i)
        sns.distplot(df[vars[i-1]].dropna())
    plt.tight_layout()
    plt.show()

In [9]:
#Visualization - boxplot
def visualization_boxplot(vars,df):
    plt.figure(figsize=(15,4))
    for i in range(1,len(vars)+1):
        plt.subplot(1,3,i)
        plt.boxplot(df[vars[i-1]].dropna())
        plt.xlabel(str(vars[i-1]))
    plt.tight_layout()
    plt.show()

In [10]:
def clean_content(text, list_to_remove):
    #xoá bỏ ký tự đặc biệt
    text = re.sub('[\-\–\,\+\?\%\/\•\*\&\[\]\(\)\:\;]', ' ', text).replace('v.',' ').replace('...',' ').replace('.',' ').replace('…',' ')
    #lowercase và tách riêng các câu phân cách bởi '\n'
    text = text.lower().split('\n')
    #loại bỏ khoảng trắng ở đầu và cuối mỗi câu
    text = [e.strip() for e in text]
    #loại bỏ các câu trùng nhau
    text = list(set(text) - set(list_to_remove))
    #loại bỏ các câu chứa 'sku'
    text = [e for e in text if 'sku' not in e]
    return text

In [11]:
def gensim_preparation(df,stop_words):
    # Tokenize(split) sentence into words
    products_gen = [[text for text in x.split()] for x in df['content_wt']]
    
    # remove some more special elements & stopwords 
    products_gen_re = [[t.lower() for t in text if not t in ['', ' ','thương hiệu','xuất xứ']] for text in  products_gen] # ký tự đặc biệt
    products_gen_re = [[t for t in text if not t in stop_words] for text in products_gen_re] # stopword
    
    # Obtain the number of features based on dicjtionary: use corpora.Dictionary
    dictionary = corpora.Dictionary(products_gen_re)
    
    # Numbers of features (word) in dictionary
    feature_cnt = len(dictionary.token2id)
    
    # Obtain corpus based on dictionary (dense matrix)
    corpus = [dictionary.doc2bow(text) for text in products_gen_re]
    
    # Use TF-IDF to process corpus, obtaining index
    tfidf = models.TfidfModel(corpus)

    # Tính toán sự tương tự trong ma trận thưa thớt
    index_sparse_matrix = similarities.SparseMatrixSimilarity(tfidf[corpus],
                                               num_features=feature_cnt)
    return tfidf, dictionary, index_sparse_matrix

In [12]:
# When user choose one product
def gensim_get_recommendation(df, tfidf, dictionary, index_sparse_matrix, product_id):
    """
    Gợi ý 5 sản phẩm có độ tương đồng cao nhất 
    """
    # sản phẩm đang xem
    name_description_pre = df[df['item_id']==product_id]['content_wt'].iloc[0]
    view_product = name_description_pre.lower().split()

    # convert search words into Sparse Vectors
    kw_vector = dictionary.doc2bow(view_product)

    # similarity calculation
    sim = index_sparse_matrix[tfidf[kw_vector]]
    
    # 5 sản phẩm có sim cao nhất (không tính sản phẩm chọn có index -1)
    indexes = sim.argsort()[-6:-1]
    result = df.iloc[:,0:9].iloc[sim.argsort()[-6:-1]]
    result['sim_score'] = np.sort(sim)[-6:-1]
    print('*** SELECTED PRODUCT ***')
    display(df.iloc[:,0:9][df['item_id']==product_id])
    print('*** 5 RECOMMENDED PRODUCTS ***')
    return result.sort_values('sim_score', ascending=False)

In [13]:
def cosine_preparation(df, stop_words):
    """
    Dùng hàm "TfidfVectorizer" để chuẩn hóa "content_wt" với:
    + analyzer='word': chọn đơn vị trích xuất là word
    + ngram_range=(1, 1): mỗi lần trích xuất 1 word
    + min_df=0: tỉ lệ word không đọc được là 0
    Lúc này ma trận trả về với số dòng tương ứng với số sp và số cột tương ứng với số từ được tách ra từ "content_wt"
    """
    tf = TfidfVectorizer(analyzer='word', stop_words=stop_words, min_df=0)
    tfidf_matrix = tf.fit_transform(df['content_wt'])

    """
    Dùng hàm "linear_kernel" để tạo thành ma trận hình vuông với số hàng và số cột là số lượng sp
    để tính toán điểm tương đồng giữa từng sp với nhau
    """
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_sim

In [14]:
# với mỗi sản phẩm, lấy 5 sản phẩm tương quan nhất
def cosine_recommendation(df,cosine_similarities,product_id):
    results = {}
    select_product_idx = df[df['item_id'] == product_id].index[0]
    for idx, row in df.iterrows():    
        similar_indices = cosine_similarities[idx].argsort()[-6:-1]
        similar_items = [(cosine_similarities[idx][e]) for e in similar_indices]
        similar_items = [(cosine_similarities[idx][e], df.index[e]) for e in similar_indices]
        results[idx] = similar_items[0:]
    print('*** SELECTED PRODUCT ***')
    display(df.iloc[:,0:9][df['item_id']==product_id])
    print('*** 5 RECOMMENDED PRODUCTS ***')
    rec_result = df.iloc[:,0:9].iloc[[e[1] for e in results[select_product_idx]]]
    rec_result['consine_sim_score'] = [e[0] for e in results[select_product_idx]]
    return rec_result.sort_values('consine_sim_score', ascending=False)

In [15]:
def ALS_evaluate(model, test_data):
    predictions = model.transform(test_data)
    evaluator = RegressionEvaluator(metricName='rmse',
                               labelCol='rating',
                               predictionCol='prediction')
    rmse = evaluator.evaluate(predictions)
    return 'RMSE = ' + str(rmse)

In [16]:
def ALS_get_recommendation(model, indexed_df, customer_id_num):
    user_recs = model.recommendForAllUsers(5)

    # create a df of distinct 'customer_id_idx' & 'customer_id' and join to user_recs
    df_reviewer_reviewer_id = indexed_df.select('customer_id_idx', 'customer_id').distinct()
    new_user_recs = user_recs.join(df_reviewer_reviewer_id, on='customer_id_idx', how='left')

    # create a df of distinct 'product_id_idx' & 'product_id'
    df_product_product_idx = indexed_df.select('product_id_idx', 'product_id').distinct()

    # Recommendation for a customer_id_num
    find_user_rec = new_user_recs.filter(new_user_recs['customer_id'] == customer_id_num)
    user = find_user_rec.first() 
    lst = []
    for row in user['recommendations']:
        row_f = df_product_product_idx.filter(df_product_product_idx.product_id_idx == row['product_id_idx'])
        row_f_first = row_f.first()
        lst.append((row['product_id_idx'], row_f_first['product_id'], row['rating']))
    dic_user_rec = {'customer_id' : user.customer_id, 'recommendations' :lst}
    rec_product_id_lst = [x[1] for x in lst]
    return dic_user_rec, rec_product_id_lst